In [15]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, lit, udf, when, isnan, count, countDistinct, desc, asc, row_number, monotonically_increasing_id
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# import the data (chunksize returns jsonReader for iteration)
businesses = pd.read_json("Data/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=100000)
reviews = pd.read_json("Data/yelp_academic_dataset_review.json", lines=True, orient='columns', chunksize=100000)

In [4]:
# read the data
for business in businesses:
    business_chunk = business
    break

for review in reviews:
    review_chunk = review
    break

In [5]:
business_subset = business_chunk[['business_id','name','address', 'categories', 'attributes','stars']]
business_subset = business_subset[business_subset['categories'].str.contains('Restaurant.*')==True].reset_index()

In [6]:
df_review = review_chunk[['user_id','business_id','stars', 'date']]

In [9]:
df_restaurant = business_subset[['business_id', 'name', 'address']]
df_restaurant.head()

,business_id,name,address
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,


In [11]:
all_combined = pd.merge(df_review, df_restaurant, on='business_id')
all_combined.head()

,user_id,business_id,stars,date,name,address
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11,Turning Point of North Wales,1460 Bethlehem Pike
1,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2,2017-05-13 17:06:55,Turning Point of North Wales,1460 Bethlehem Pike
2,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,4,2017-08-08 00:58:18,Turning Point of North Wales,1460 Bethlehem Pike
3,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,3,2017-11-19 02:20:23,Turning Point of North Wales,1460 Bethlehem Pike
4,dCooFVCk8M1nVaQqcfTL3Q,XQfwVwDr-v0ZS3_CbbE5Xw,2,2017-09-09 17:49:47,Turning Point of North Wales,1460 Bethlehem Pike


In [12]:
all_combined.shape

(72125, 6)

In [13]:
# create a user-item matrix
rating_crosstab = all_combined.pivot_table(values='stars', index='user_id', columns='name', fill_value=0)
rating_crosstab.head()

name,'feine,101 Taiwanese Cuisine,10th Street Italian,1200 Chophouse,12th & Porter,16th Street Seafood,1911 Smoke House Barbeque,1925 Cocktail Lounge,2 in One Cafe,211 York,...,eegee's,fat Rooster diner,honeygrow,iCafe,iLuv Pho,iPho Vietnamese Restaurant,il Tavolo Trattoria,la Madeleine,swah-rey,sweetgreen
user_id,,,,,,,,,,,,,,,,,,,,,
---2PmXbF47D870stH1jqA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--4AjktZiHowEIBCMd4CZA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--E0uVPphTORm_OiZ5KCvA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--KMTwCrhKKUmr7riuS4WQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--S8M395r8NtOCvS2LRfDw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Transpose the Utility matrix
X = rating_crosstab.values.T

In [16]:
def calculate_similarity(user_item_matrix):
    """
    Calculate the cosine similarity matrix from the user-item matrix
    """
    similarity = cosine_similarity(user_item_matrix)
    np.fill_diagonal(similarity, 0)
    return pd.DataFrame(similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Prediction of Ratings 

def predict_ratings(similarity, user_item_matrix, user_id):
    """
    Predict ratings for all items for a given user
    """
    total_similarity = similarity[user_id].sum()
    weighted_sum = np.dot(similarity[user_id], user_item_matrix.fillna(0))

    # Avoid division by zero
    if total_similarity == 0:
        total_similarity = 1

    predictions = weighted_sum / total_similarity
    predictions = pd.Series(predictions, index=user_item_matrix.columns)
    return predictions

def train_test_split_and_predict(data):
    """
    Split the data into train and test sets, predict ratings, and return the true and predicted ratings
    """
    train_user_item_matrix, test_user_item_matrix = train_test_split(data, test_size=0.02)

    similarity = calculate_similarity(train_user_item_matrix)
    true_ratings = []
    pred_ratings = []

    for user_id in test_user_item_matrix.index:
        if user_id in test_user_item_matrix.index:
            true_rating = test_user_item_matrix.loc[user_id]
            pred_rating = predict_ratings(similarity, train_user_item_matrix, user_id)
            true_ratings.extend(true_rating[true_rating.notnull()])
            pred_ratings.extend(pred_rating[true_rating.notnull()])

    return true_ratings, pred_ratings

def evaluate_performance(data):
    """
    Evaluate the performance of the collaborative filtering algorithm
    """
    true_ratings, pred_ratings = train_test_split_and_predict(data)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))
    mae = mean_absolute_error(true_ratings, pred_ratings)
    return rmse, mae

In [17]:
rmse, mae = evaluate_performance(rating_crosstab)
print(f'RMSE: {rmse}, MAE: {mae}')

KeyError: 'FkaBNfCc2dRyQ2qXhsXXtg'